In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

In [74]:
import os 
import playsound
import time
import speech_recognition as sr
from gtts import gTTS
import pyttsx3
import pytz
import subprocess
import webbrowser

In [4]:
MONTHS=["january","february","march","april","may","june","july","august","september","october","november","december"]
DAYS=["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
DAYS_EXTENSIONS=["rd","th","st","nd"]



counter=3
name=""
#maintain this counter bcoz of this 
#PermissionError: [Errno 13] Permission denied: 'voice2.mp3' so every time this function calls we create new file

In [5]:
def speak(text):
    engine=pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [6]:
speak("hello nitin sharma")

In [66]:
#getting input from the user
def get_audio():
    r=sr.Recognizer()
    with sr.Microphone() as source:
        audio=r.listen(source)
        said=""
        try:
            said=r.recognize_google(audio)
            print(said)
        except Exception as e:
            print(str(e))
    return said.lower()

In [46]:
text=get_audio()
if "hello" in text:
    speak("hello how are you")
    
elif "good morning" in text:
    speak("good morning have a good day")

good morning


In [67]:
def authenticate_google():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    
    return service

In [68]:
def get_events(day,serivce):
    # Call the Calendar API
    date =datetime.datetime.combine(day,datetime.datetime.min.time())#give the minimum time for the particular day
    end_date_time =datetime.datetime.combine(day,datetime.datetime.max.time())
    
    utc=pytz.UTC
    date=date.astimezone(utc)
    end_date_time=end_date_time.astimezone(utc)
    
    
    
    events_result = service.events().list(calendarId='primary', timeMin=date.isoformat(),timeMax=end_date_time.isoformat(),
                                        singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        speak('No upcoming events found.')
    else:
        speak("you have {} event on this day".format(len(events)))
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])
            
            start_time=str(start.split("T")[1].split("-")[0]).split(":")[0]
            print(start_time)
            
            if int(start_time.split(":")[0])>12:
                start_time=start_time+"am"
            else:
               
                start_time=start_time+"pm"
            speak(event["summary"]+"at"+start_time)

In [69]:
#creating our own function to find out the future event we have by asking the assisstant
def get_date(text):
    text=text.lower() #converting the text into lower form for the ease
    today=datetime.date.today() #find out the today's date
    
    
    if text.count("today") > 0: #if the user is talking about current day event so we return the current date
        return today
    
    
    day=-1
    day_of_week=-1
    month=-1
    year=today.year
    
    for word in text.split():
       
        if word in MONTHS: #checking for the month
            month=MONTHS.index(word)+1 
            
        elif word in DAYS:#checking for the day
            day_of_week=DAYS.index(word)
        elif word.isdigit():#checking is the word is number or not
            day=int(word)
        else: #checking of extensions like 6th feb like this
            for ext in DAYS_EXTENSIONS:
                
                found=word.find(ext) #this will give the starting index like 6th so extension starts from (0=6,1=t,2=h index of
                # of the word so it will return 1)
                if found>0:
                    try:
                        day=int(word[:found])
                        print("the day"+day)#so we slice the word till the extension appear for example 6th so we slice to get 6
                    except:
                        pass
    if month < today.month and month!=-1: #if the month is less than today month means we ask about the month of next year
        #ex if today's month is august and we ask about feb to it means feb of next year 
        year=year+1
    if day < today.day and month==-1 and day!=-1: #what we did above same thing we are doing here
        month=month+1

    if month==-1 and day==-1 and day_of_week!=-1:
        current_day_of_week=today.weekday() #this function will give us the value of the current day ex for mon=0 and tue=1
        #and for sun=6 (mon,------,sun)=(0,-----,6)
        dif=day_of_week-current_day_of_week

        if dif < 0:#if the difference is negative that means we are talking about next week for ex today is sunday
            #so cur_day=6 and if we say what is the plan we have on tuesday so day_of_week=1 the diff is 1-6 ie -5 which
            #is negative and it is obvious if today is sunday and we are talking about tuesday it must be of next week
            #so we add +7 to dif ie -5+7=2
            dif+=7
            if text.count("next")>=1: #here we check how many time next occur if the sentence is something like next to next 
                #this means we are talking about next to next week
                dif+=7

        return today + datetime.timedelta(dif)#timedelta function add dif with current day
    if month==-1 or day ==-1:
        return None

    return datetime.date(month=month,day=day,year=year)    

In [70]:
#function for creating notes
def notes(text):
    date=datetime.datetime.now() #date and time for the current day
    file_name=str(date).replace(":","-")+"-note.txt"
    with open(file_name,"w") as f:
        f.write(text)
        
    subprocess.Popen(["notepad.exe",file_name])

In [87]:
Wake="hi nitin" #this is use for starting our bot
stop="bye"
google_path="C:/Users/NITIN/AppData/Local/Google/Chrome/Application/chrome.exe %s"
service=authenticate_google()
print("start")

while True:
    print("Listening")
    text=get_audio() #here we take audio for activating the bot
    
    if text.count(Wake) > 0:
        speak("i am ready")
        
        text=get_audio() #here we take audio for working purpose
        CALENDAR_STRS=["what do i have","do i have plans","am i busy"] #these are string that is use whenever we have to ask for some events
        #we make these strings for the optimization purpose we dont want that out bot starts giving us responses with the sentences which 
        #dont have any meaning
        for phrase in CALENDAR_STRS:
            if phrase in text:
                date=get_date(text)
                if date:
                    get_events(date,service)
                else:
                    speak("i dont understand")
        NOTES_STRS=["make a note","write this down","remember this"]
        for phrase in NOTES_STRS:
            if phrase in text:
                speak("what would you like me to write down")
                note_text=get_audio()
                notes(note_text) #calling note function
                speak("i have made a note")
                
        if "open google" in text:
            speak("ok wait i am opening for you")
            webbrowser.get(google_path).open_new("https://www.google.com")
            
        if "open stackoverflow" in text:
            speak("ok wait i am opening for you")
            webbrowser.get(google_path).open_new("stackoverflow.com")

    if text.count(stop) > 0:
        speak("thank you have a nice day")
        break

start
Listening
hi Nitesh
Listening
hi Nitin
open Google
Listening

Listening
bye
